In [1]:
#引入库
import requests
import logging
import re
from urllib.parse import urljoin


In [12]:
#定义变量
logging.basicConfig(level=logging.INFO,format='%(asctime)s-%(levelname)s:%(message)s')
BASE_URL='https://ssr1.scrape.center'
TOTAL_PAGE=10

In [4]:
#方法
def scrape_page(url):
    logging.info('scraping %s...',url)
    try:
        response=requests.get(url)
        if response.status_code==200:
            return response.text
        logging.error('get invalid status code %s while scraping %s',response.status_code,url)

    except requests.RequestException:
        logging.error('error occurred while scraping %s',url,exc_info=True)

In [5]:
#列表爬取方法
def scrape_index(page):
    index_url=f'{BASE_URL}/page/{page}'
    return scrape_page(index_url)


In [8]:
#pares_方法
def parse_index(html):
    pattern=re.compile('<a.*?href="(.*?)".*?class="name">')
    items=re.findall(pattern,html)
    if not items:
        return []
    for item in items:
        detail_url=urljoin(BASE_URL,item)
        logging.info('get detail url %s',detail_url)
        yield detail_url


In [13]:
#串联
def main():
    for page in range(1,TOTAL_PAGE+1):
        index_html=scrape_index(page)
        detail_urls=parse_index(index_html)
        logging.info('detail urls %s',list(detail_urls))
if __name__=='__main__':
    main()


2025-09-08 14:05:50,985-INFO:scraping https://ssr1.scrape.center/page/1...
2025-09-08 14:05:51,544-INFO:get detail url https://ssr1.scrape.center/detail/1
2025-09-08 14:05:51,560-INFO:get detail url https://ssr1.scrape.center/detail/2
2025-09-08 14:05:51,560-INFO:get detail url https://ssr1.scrape.center/detail/3
2025-09-08 14:05:51,560-INFO:get detail url https://ssr1.scrape.center/detail/4
2025-09-08 14:05:51,560-INFO:get detail url https://ssr1.scrape.center/detail/5
2025-09-08 14:05:51,583-INFO:get detail url https://ssr1.scrape.center/detail/6
2025-09-08 14:05:51,587-INFO:get detail url https://ssr1.scrape.center/detail/7
2025-09-08 14:05:51,591-INFO:get detail url https://ssr1.scrape.center/detail/8
2025-09-08 14:05:51,593-INFO:get detail url https://ssr1.scrape.center/detail/9
2025-09-08 14:05:51,595-INFO:get detail url https://ssr1.scrape.center/detail/10
2025-09-08 14:05:51,596-INFO:detail urls ['https://ssr1.scrape.center/detail/1', 'https://ssr1.scrape.center/detail/2', 'htt

In [14]:
#定义爬取方法
def scrape_detail(url):
    return scrape_page(url) 

In [17]:
#解析详情页
def parse_detail(html):
    cover_pattren=re.compile('class="item.*?<img.*?src="(.*?)".*?class="cover">',re.S)
    name_pattern=re.compile('<h2.*?>(.*?)</h2>')
    categories_pattern=re.compile('<button.*?category.*?<span>(.*?)</span>.*?</button>',re.S)
    published_at_pattern=re.compile(r'(\d{4}-\d{2}-\d{2})\s?上映')
    drama_pattern = re.compile('<div.*?drama.*?<p.*?>(.*?)</p >', re.S)
    score_pattern = re.compile(r'<p.*?score.*?>(.*?)</p >', re.S)
    
    cover = re.search(cover_pattern, html).group(1).strip() if re.search(cover_pattern, html) else None
    name = re.search(name_pattern, html).group(1).strip() if re.search(name_pattern, html) else None
    categories = re.findall(categories_pattern, html) if re.search(categories_pattern, html) else []
    published_at = re.search(published_at_pattern, html).group(1) if re.search(published_at_pattern, html) else None
    drama = re.search(drama_pattern, html).group(1).strip() if re.search(drama_pattern, html) else None
    score = float(re.search(score_pattern, html).group(1).strip()) if re.search(score_pattern, html) else None
    
    return {
        'cover': cover,
        'name': name,
        'categories': categories,
        'published_at': published_at,
        'drama': drama,
        'score': score
    }

In [19]:
import json
from os import path
import os

RESULTS_DIR = 'results'
os.path.exists(RESULTS_DIR) or os.makedirs(RESULTS_DIR)

def save_data(data):
    name = data.get('name')
    data_path = f'{RESULTS_DIR}/{name}.json'
    with open(data_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:







import multiprocessing
import logging

RESULTS_DIR = 'results'
pages = range(1, TOTAL_PAGE + 1)      # 待爬页码序列

def main(page):
    """单页爬取 + 保存全流程"""
    index_html = scrape_index(page)               # 1. 拉取列表页
    detail_urls = parse_index(index_html)         # 2. 解析出详情页链接

    for detail_url in detail_urls:                # 3. 遍历详情页
        detail_html = scrape_detail(detail_url)   #    拉取详情页
        data = parse_detail(detail_html)          #    提取字段
        logging.info('get detail data %s', data)
        logging.info('saving data to json file')
        save_data(data)                           #    落盘为 json
        logging.info('data saved successfully')

if __name__ == '__main__':
    # 多进程加速：一页一个进程
    with multiprocessing.Pool() as pool:
        pool.map(main, pages)   # 把 pages 分给进程池
